In [9]:
import re
import json
from io import BytesIO

import requests
import pandas as pd
from glob import glob
from PIL import Image

In [2]:
ad_keywords = [
    "내돈내산",
    "지원을받아",
    "지원받아",
    "후원을받아",
    "후원받아",
    "원고료",
    "체험단",
    "제공받아",
    "금전적지원",
    "사비로",
    "업체로부터",
    "협찬",
    "제품또는서비스를",
    "업체에서",
    "소정의",
    "고료를",
    "할인을",
    "레뷰",
    "강남오빠",
    "디너의여왕",
    "서울오빠",
    "이놀자데뷰"
]

In [3]:
def load_img(url: str):
    res = requests.get(url)
    img = Image.open(BytesIO(res.content))
    
    return img

In [4]:
HAN_REGEX = re.compile('[^가-힣]')

In [10]:
files = glob('./data/semi_struct_ocr/*.json')
files

['./data/semi_struct_ocr/홍대 회식 맛집.json',
 './data/semi_struct_ocr/홍대 데이트 맛집.json',
 './data/semi_struct_ocr/성수 데이트 맛집.json']

In [11]:
with open(files[2], 'r', encoding='utf-8') as f:
    blog = json.load(f)

In [12]:
df = []
for article in blog:
    easy_text = ""
    tess_text = ""
    plain_text = ""
    for p in article['contents']:
        if p['info'] in ['img', 'gif']:
        
            if p.get('tess_text'):
                easy_text += p['easy_text']
                tess_text += p["tess_text"]
        elif p['info'] == 'text':
            plain_text += p['content']
            
    df.append({
        "article_id": article["id"],
        "plain_text": HAN_REGEX.sub('', plain_text),
        "easyocr": HAN_REGEX.sub('', easy_text),
        "tesseractocr": HAN_REGEX.sub('', tess_text)
    })
df = pd.DataFrame(df)

In [13]:
for kywd in ad_keywords:
    df.loc[df[df.plain_text.str.contains(kywd)].index, 'label'] = 1
    df.loc[df[df.tesseractocr.str.contains(kywd)].index, 'label'] = 1
    df.loc[df[df.easyocr.str.contains(kywd)].index, 'label'] = 1
df['label'] = df.label.fillna(0)

In [16]:
df[df.label == 1]['tesseractocr'].apply(lambda x: x[-20:])

0      제품또는서비스를제공받아작성된리뷰입니다
2      품원고료를지원받아솔직하게작성하였습니다
3                          
4      제공받아체험한후솔직하게작성한후기입니다
5      각의견으로자성하리뷰입니다참고만해주세요
               ...         
975    기예예린가네사케지도본도에베기예때띠르네
976    애금찌버버며팬이더비마떼은기그개버귀특고
979    서비스틀제공받아솔직하게작성하였습니다이
981    다락에서원고료를제공받아작성된글입니다주
984    서비스틀제공받아솔직하게작성하였습니다이
Name: tesseractocr, Length: 486, dtype: object